In [1]:
import os
import re
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, 
    DataCollatorForSeq2Seq, 
    EarlyStoppingCallback)

from datasets import load_metric, Dataset

import wandb
import nltk

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()

- 사전학습 모델인 ainize/kobart-news에 요약 데이터를 사용해 fine-tuning을 진행합니다.
- 생성 메소드를 실험하기 위한 훈련으로 2994개 샘플을 포함한 작은 데이터세트를 활용합니다.

# Paths and Names

- 데이터와 모델의 CKPT, 그리고 메트릭의 이름과 경로를 설정합니다.
- 훈련 생성물과 WandB 로깅을 위한 설정도 함께 진행합니다.

In [3]:
### paths and names

PROJECT_NAME = 'text_summarization'
RUN_ID = 'v2'

DATA_PATH = 'data/original/train.csv'

MODEL_CHECKPOINT = 'ainize/kobart-news'
METRIC_NAME = 'rouge'

NOTEBOOK_NAME = './train.ipynb'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, 'training_results', RUN_ID)

model_name = re.sub(r'[/-]', r'_', MODEL_CHECKPOINT).lower()
run_name = f'{model_name}_{RUN_ID}'
output_dir = os.path.join(SAVE_PATH, run_name)

!mkdir -p {SAVE_PATH}

In [4]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find {NOTEBOOK_PATH}.


env: WANDB_PROJECT=text_summarization
env: WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin


True

# Training Args

- 훈련에 적용될 HP와 각종 변수를 설정해줍니다.
- 3e-6 * NGPU(4)의 LR로 30 에폭 훈련을 진행할 것입니다. BS는 GPU당 4로 할당합니다.
- 옵티마이저는 pytorch의 adamw 클래스를 불러오며, LR Scheduler는 Cosine Annealing Schedule를 사용합니다.
- 모델은 Evaluation Loss를 기준으로 Best와 Last가 각각 저장됩니다.
- 손실값과 평가지표는 모두 Greedy Search를 통한 출력값으로 계산됩니다.

In [5]:
report_to="wandb"

num_train_epochs = 30
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1

optim = 'adamw_torch' # 'adamw_torch' or 'adamw_hf'

learning_rate = 3e-6 * NGPU
weight_decay = 0.01
adam_epsilon = 1e-8

lr_scheduler_type = 'cosine' # 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup'
warmup_ratio = 0

save_total_limit = 2

load_best_model_at_end = True
metric_for_best_model = 'eval_loss'

save_strategy = "epoch"
evaluation_strategy = "epoch"

logging_strategy = "steps"
logging_first_step = True 
logging_steps = int(500 / NGPU)

predict_with_generate=True
generation_max_length=512
# generation_num_beams=5

fp16 = False

# Model & Tokenizer & Metric

- 모델과 토크나이저, 그리고 평가지표를 계산하는 데 사용할 함수를 불러옵니다.
- 모델의 config에는 사용하지 않는 설정이 포함되어 있습니다. 삭제합니다.

In [6]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)
del config.label2id
del config.id2label
del config.task_specific_params

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
metric = load_metric(METRIC_NAME)

# Functions

- 모델 훈련을 위해 전처리를 수행해주는 함수를 선언합니다.
- 입력 문장이 input이 되고 요약문이 label이 됩니다.
- tokenizer를 사용해 input_ids로 변환하고 입력값에는 attention_mask를 생성해줍니다.
- 입력 문장에 prefix를 추가하여 성능 실험을 해볼 수 있으나 지금은 적용하지 않겠습니다.

In [8]:
prefix = ""

max_input_length = 1026
max_target_length = 514

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["context"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

- 모델의 예측값과 위에서 전처리한 라벨을 활용하여 평가지표를 출력하는 함수를 선언합니다.
- ROUGE를 평가지표로 사용합니다.
    - ROUGE-N (N-gram) scoring
    - ROUGE-L (Longest Common Subsequence) scoring
        - sentence-level: Compute longest common subsequence (LCS) between two pieces of text. Newlines are ignored. This is called rougeL in this package.
        - summary-level: Newlines in the text are interpreted as sentence boundaries, and the LCS is computed between each pair of reference and candidate sentences, and something called union-LCS is computed. This is called rougeLsum in this package.

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# Inputs and Labels

- csv로 저장된 데이터를 불러와 9:1로 훈련과 검증에 사용합니다.
- 선언한 전처리 함수를 적용하여 데이터세트를 생성합니다.

In [10]:
data_df = pd.read_csv(DATA_PATH)

In [11]:
dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.1)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [12]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=NCPU, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=NCPU, 
                                remove_columns=eval_dataset.column_names)
print(train_dataset)
print(eval_dataset)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#20:   0%|          | 0/1 [00:00<?, ?ba/s]

#21:   0%|          | 0/1 [00:00<?, ?ba/s]

#22:   0%|          | 0/1 [00:00<?, ?ba/s]

#23:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 2694
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 300
})


# Train

- 상단에서 선언한 HP와 각종 변수들로 training_args를 생성합니다.

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,

    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,

    predict_with_generate=predict_with_generate,
    generation_max_length=generation_max_length,
    # generation_num_beams=generation_num_beams,

    fp16=fp16,
)

- 배치 단위로 padding처리를 하여 훈련 속도를 높히기 위해 data_collator를 선언합니다.
- 생성한 객체들을 trainer에 포함시켜 훈련 준비를 마칩니다.

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Seq2SeqTrainer(
    model=model,
    
    args=training_args,
    
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    
    tokenizer=tokenizer,
    data_collator=data_collator,
    
    compute_metrics=compute_metrics,
)

- 훈련을 시작합니다.

In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2694
  Num Epochs = 30
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5070
  Number of trainable parameters = 123859968
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.382300,0.934572,55.098000,34.948400,55.008900,54.892600,42.080000
2,0.873900,0.857066,57.091300,36.787900,56.941400,56.846900,40.320000
3,0.663000,0.839307,59.083500,39.600000,59.031300,58.936700,43.120000
4,0.577900,0.829521,60.400800,40.113500,60.263000,60.105700,41.383300
5,0.505900,0.841939,60.153900,41.184400,60.028800,59.616800,42.340000
6,0.439100,0.858681,59.787200,40.521900,59.770600,59.387600,42.430000
7,0.392000,0.889838,60.544800,40.215400,60.541500,60.199700,43.866700
8,0.346300,0.911681,61.603300,41.226500,61.521600,61.344900,42.450000
9,0.293200,0.920116,59.913100,40.344600,59.871500,59.566100,43.140000
10,0.267900,0.957089,61.320600,40.701600,61.079800,61.060400,40.733300


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16
Saving model checkpoint to ./training_results/v2/ainize_kobart_news_v2/checkpoint-169
Configuration saved in ./training_results/v2/ainize_kobart_news_v2/checkpoint-169/config.json
Model weights saved in ./training_results/v2/ainize_kobart_news_v2/checkpoint-169/pytorch_model.bin
tokenizer config file saved in ./training_results/v2/ainize_kobart_news_v2/checkpoint-169/tokenizer_config.json
Special tokens file saved in ./training_results/v2/ainize_kobart_news_v2/checkpoint-169/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token

TrainOutput(global_step=5070, training_loss=0.25611141423972167, metrics={'train_runtime': 8032.2127, 'train_samples_per_second': 10.062, 'train_steps_per_second': 0.631, 'total_flos': 4.906335038072832e+16, 'train_loss': 0.25611141423972167, 'epoch': 30.0})

- 옵티마이저 스테이트를 포함한 훈련 재개에 사용할 파일들을 삭제해줍니다.
- 추후 불러올 모델과 토크나이저 관련 파일만 남겨둡니다.

In [16]:
keep = [
    'added_tokens.json',
    'config.json',
    'pytorch_model.bin',
    'special_tokens_map.json',
    'tokenizer.json',
    'tokenizer_config.json',
    'vocab.txt'
]

ckpts = os.listdir(output_dir)
for ckpt in ckpts:
    ckpt = os.path.join(output_dir, ckpt)
    for item in os.listdir(ckpt):
        if item not in keep:
            os.remove(os.path.join(ckpt, item))

# Generate

- 훈련을 마치면 Evaluation Loss 기준 Best 모델이 로드되어 있습니다.
- trainer는 Greedy Search를 수행하도록 설정되어 있습니다.
- Evaluation 데이터를 활용해 모델의 출력을 간단히 살펴봅니다.

In [17]:
preds = trainer.predict(eval_dataset)

The following columns in the test set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 300
  Batch size = 16


In [18]:
for data, pred in zip(eval_dataset, preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')

정답: 제181회 완주군의회 임시회 회의록 서명의원으로 이향자 의원과 송형중 의원이 선출됨.
예측: 제181회 완주군의회 임시회 회의록 서명의원으로 이향자 의원과 송현중 의원이 선출됨.

정답: 7월 12일부터 7월 20일까지 9일간 본회의 휴회가 가결됨. 제2차 본회의는 7월 21일 오전 10시에 개의.
예측: 7월 12일부터 7월 20일까지 9일간 휴회가 가결됨. 제2차 본회의는 7월 21일 오전 10시에 개의.

정답: 음성군 농촌전문인력육성기금 운용관리 조례 폐지조례안은 이율 감소로 인한 재원고갈 우려, 기금 유지의 어려움으로 인해 <지방자치단체 기금관리기본법>제15조의 규정에 따라 제안되었음. 음성장학회 사업으로는 대체가 불가능하여 폐지조례안은 필요치 않다는 의견이 나왔으며, 기금을 폐지하더라도 장학금 유지를 촉구함. 해당 안건은 찬성 6, 반대 1로 가결됨.
예측: 음성군 농촌전문인력육성기금 사업은 이자율의 감소로 재원 고갈이 우려되며, 추진사업 분야가 장학금 지급으로 한정되어 있어 계속적으로 기금을 유지하는데 어려움이 있어 폐지하고자 하는 것으로 <지방자치단체 기금관리기본법>에 따라 조례를 폐지하고자 함. 해당 안건은 가결됨.

정답: 음성군공무원직장협의회의설립, 운영에관한조례제정조례안은 공무원직장협의회의설립, 운영에관한법률 시행에 따라 동법률에서 위임된 직장협의회의 설립 및 운영에 관한 기본적인 사항과 시행에 필요한 사항을 조례로 제정하기 위해 상정됨. 해당 안건은 가결됨.
예측: 음성군 공무원직장협의회의 설립 및 운영에 관한 조례제정조례안은 공무원의 근무환경개선, 업무능률 및 고충처리 등을 목적으로 6급 이하 공무원들이 설립, 운영하는 공무원직장협의회의설립, 운영에관한법률이 시행됨에 따라 동법률에서 위임된 직장협의회의 설립 및 운영에 관한 기본적인 사항과 그 시행에 필요한 사항을 조례로 제정하고자 제안됨. 해당 안건은 가결됨.

정답: 2019년도 예산안 심사결과 보고는 가결됨.
예측: 2019년도 예산안은 가결됨. 2019년도 수정예산안은 가